## Importing libraries

In [185]:
import sys
sys.path.append("I:/My Drive/suyana")
from scripts.utils.regression_functions import *

from shapely import Polygon, LineString, Point

In [147]:
from scipy.io import netcdf_file
import xarray as xr
import requests
from scipy.stats import norm

In [128]:
import os
path = "I:/My Drive/suyana/data/capacity/"
files = os.listdir(path)
files = [f for f in files if f.endswith(".nc")]

id_canton = [files[i].split("_")[1].split(".")[0] for i in range(len(files))]

In [129]:
list_dates = []
list_swc = []
list_id = []
data = pd.DataFrame()
for file in files:
    id_canton = file.split("_")[1].split(".")[0]
    netcdf_file = xr.open_dataset(path + file)
    
    data_aux = pd.DataFrame({
        "date": netcdf_file.time.values,
        "swc": netcdf_file.swc_adjusted.values
    })
    data_aux["idCanton"] = id_canton

    data = pd.concat([data, data_aux], axis=0)

## Exploratory Data Analysis

In [62]:
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month

In [140]:
netcdf_file.variables

Frozen({'swc_raw': <xarray.Variable (time: 8176)> Size: 65kB
array([     nan, 0.308279, 0.2454  , ..., 0.320967, 0.319731,      nan],
      shape=(8176,)), 'week': <xarray.Variable (time: 8176)> Size: 65kB
[8176 values with dtype=float64], 'swc_adjusted': <xarray.Variable (time: 8176)> Size: 65kB
array([     nan, 0.350339, 0.269238, ..., 0.320967, 0.319731,      nan],
      shape=(8176,)), 'time': <xarray.IndexVariable 'time' (time: 8176)> Size: 65kB
array(['2002-06-15T00:00:00.000000000', '2002-06-16T00:00:00.000000000',
       '2002-06-17T00:00:00.000000000', ..., '2024-10-29T00:00:00.000000000',
       '2024-10-30T00:00:00.000000000', '2024-10-31T00:00:00.000000000'],
      shape=(8176,), dtype='datetime64[ns]')})

In [64]:
data_summary = data.groupby(["idCanton", "year"]).size().reset_index()

In [65]:
## CANTIDAD DE OBSERVACIONES POR AÑO
data_summary.pivot(index="idCanton", columns="year", values=0)

year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
idCanton,,,,,,,,,,,,,,,,,,,,,,,
43,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
45,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
49,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
50,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
51,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
52,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
53,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
54,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305
55,200,365,366,365,365,365,366,365,365,365,367,365,365,365,366,365,365,365,366,365,365,365,305


In [70]:
## DUPLICADOS
data[data.year==2012].groupby(["date", "idCanton"]).agg(n=("idCanton","count")).reset_index().query('n>1')

,date,idCanton,n
3485,2012-07-24,43,2
3486,2012-07-24,45,2
3487,2012-07-24,49,2
3488,2012-07-24,50,2
3489,2012-07-24,51,2
3490,2012-07-24,52,2
3491,2012-07-24,53,2
3492,2012-07-24,54,2
3493,2012-07-24,55,2
3494,2012-07-24,56,2


In [74]:
n_anios = len(data['year'].unique())
min_year = min(data['year'])
max_year = max(data['year'])
print(f'Cantidad de años: {nAnios} desde {min_year} hasta {max_year}')
n_dates = len(data['date'].unique())
min_date = min(data['date'])
max_date = max(data['date'])
print(f'Cantidad de días: {n_dates} desde {min_date} hasta {max_date}')

Cantidad de años: 23 desde 2002 hasta 2024
Cantidad de días: 8175 desde 2002-06-15 00:00:00 hasta 2024-10-31 00:00:00


## Data from gpkg cantones bolivia

In [174]:
df_cantones = gpd.read_file('I:/My Drive/suyana/data/drought/bolivia_cantones.gpkg')

In [221]:
df_cantones.head(4)

,ogc_fid,entity,canton,provincia,municipio,departamen,geometry,contine
0,1,Polyline,Luquisani,Muńecas,Chuma,La Paz,"MULTIPOLYGON (((-68.73317 -15.52315, -68.72776...",False
1,13,Polyline,Villa Amacari,Manco Kapac,San Pedro de Tiquina,La Paz,"MULTIPOLYGON (((-68.87332 -16.27777, -68.87397...",False
2,14,Polyline,None,Los Andes,Puerto Perez,La Paz,"MULTIPOLYGON (((-68.70706 -16.36251, -68.70781...",False
3,15,Polyline,None,Los Andes,Puerto Perez,La Paz,"MULTIPOLYGON (((-68.771 -16.3759, -68.77147 -1...",False


## Data from API for yields panel data

In [158]:
# data_folder = Path("../Data/data_output_rendimiento")
# filename = "df_rendimiento_flagged_ONI.csv"
data_folder = "I:/My Drive/suyana/data/drought/"
filename = "yields_panel.csv"
df_yields = pd.read_csv(data_folder + filename)
df_yields.drop(['flag','notes', 'Unnamed: 10'], axis=1, inplace = True)

In [159]:
df_yields.head(4)

,producer_id,zone,lot,plot,orig_crop,variety,seedtime,harvest_date,show_furrow,n_plants_m,n_plants_ha,date_beginning_flowering,mature_maturation_date,plant_height,fertitilized,type_of_fertilizer,dose_kg_ha,rend_ton_ha,cond_climaticas_de_la_campana,producer_handling,id_lot,cultivation_id,culture,yr,summer,...,pp_anomaly_seed_date_minus_2_month,vhi_anomaly_seed_date_minus_2_month,ndvi_anomaly_flowering_date,ndpi_anomaly_flowering_date,pp_anomaly_flowering_date,vhi_anomaly_flowering_date,ndvi_flowering_date,ndpi_flowering_date,pp_flowering_date,vhi_flowering_date,ndvi_anomaly_flowering_date_minus_1_month,ndpi_anomaly_flowering_date_minus_1_month,pp_anomaly_flowering_date_minus_1_month,vhi_anomaly_flowering_date_minus_1_month,ndvi_anomaly_flowering_date_minus_2_month,ndpi_anomaly_flowering_date_minus_2_month,pp_anomaly_flowering_date_minus_2_month,vhi_anomaly_flowering_date_minus_2_month,Month,Year,ENSO,pixel_id,anomaly_sum,swc_sum,climatology_sum
0,16,Norte,H-2,Las Londras,Sorgo,Timbo,NaN,2016-09-13,0.45,11.0,244444.440,45,110,1.1,MAP,11-52-0,100,1.000000,NaN,NaN,67,2,SORGO,2016,0,...,-147.570695,-931.913044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2016.0,ninia,0.0,0.000000,0.0,0.000000
1,14,Este,B04,Europa,Maiz,Maiz 355,2013-03-10,2013-08-20,0.42,2.4,57142.857,5/14/2013,7/13/2013,2.1,0,0,0,1.994742,NaN,NaN,45,1,MAIZ,2013,0,...,-318.040930,-1266.048243,-573.160520,1.054872,7.770585,-25.339241,6219.065355,1.120897,18.280309,27.384402,-22986.655761,-4.350322,-99.537446,-883.337060,-36100.439166,1.791396,-109.046194,-1606.263388,8.0,2013.0,NaN,0.0,10141.265152,25988.0,15846.734848
2,14,Este,A09,Europa,Sorgo,DAS-5000,2013-03-19,2013-07-24,0.42,9.0,214285.710,5/3/2013,7/7/2013,1.1,0,0,0,2.323631,NaN,NaN,43,2,SORGO,2013,0,...,-252.626529,-1279.054223,1549.200627,0.287106,0.638863,-27.786188,6749.498087,0.222299,9.864622,24.390516,22223.058567,11.863377,107.183678,-914.857636,-14995.596168,0.469207,-157.261620,-1528.424227,7.0,2013.0,NaN,0.0,8117.090909,20801.0,12683.909091
3,12,Este,15-N,AgroOriente,Trigo,A9829R,2013-03-31,2013-08-09,0.17,50.0,2941176.500,5/9/2013,7/13/2013,NaN,0,0,0,NaN,Malo,Regular,27,5,TRIGO,2013,0,...,-503.627799,-390.735870,1530.561548,-0.418810,-6.299874,-9.022686,7196.695431,-0.459921,4.003364,38.563123,7854.427149,4.080776,32.109304,-470.953482,-18272.315065,1.485035,-102.000497,-793.850622,8.0,2013.0,NaN,0.0,8249.757576,20998.0,12748.242424


In [165]:
df_yields.columns

Index(['producer_id', 'zone', 'lot', 'plot', 'orig_crop', 'variety',
       'seedtime', 'harvest_date', 'show_furrow', 'n_plants_m', 'n_plants_ha',
       'date_beginning_flowering', 'mature_maturation_date', 'plant_height',
       'fertitilized', 'type_of_fertilizer', 'dose_kg_ha', 'rend_ton_ha',
       'cond_climaticas_de_la_campana', 'producer_handling', 'id_lot',
       'cultivation_id', 'culture', 'yr', 'summer', 'latitude', 'longitude',
       'flowering_date', 'seed_date', 'diff_date', 'closest_date_index_ndvi',
       'closest_date_index_ndpi', 'closest_date_index_pp',
       'closest_date_index_vhi', 'closest_id_grid', 'ndvi', 'ndpi', 'pp',
       'vhi', 'ndvi_anomaly', 'ndpi_anomaly', 'pp_anomaly', 'vhi_anomaly',
       'ndvi_anomaly_minus_1_month', 'ndpi_anomaly_minus_1_month',
       'pp_anomaly_minus_1_month', 'vhi_anomaly_minus_1_month',
       'ndvi_anomaly_minus_2_month', 'ndpi_anomaly_minus_2_month',
       'pp_anomaly_minus_2_month', 'vhi_anomaly_minus_2_month',
     

In [171]:
list_ids = pd.read_csv("I:/My Drive/suyana/data/available_areas.csv")
list_ids = list(list_ids['id_area'])
list_ids.sort()
list_ids[:2]

[43, 45]

## Get Full Data from API

In [169]:
API_URL = "https://api.suyana.io/drought-bolivia/time-series"
API_KEY = "93a52c7a17676ed586c058e9f5315c9d66b256affd6d955897c7653d71d7fc08"
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

In [170]:
df_full = pd.DataFrame()
for id in list_ids:
    try:
        print(id)
        params = {
            "area": id,  # Climatic area identifier
            "format": "json"  # Output format (optional, default is csv)
        }
        response = requests.get(API_URL, headers=headers, params=params)
        df = pd.json_normalize(response.json())
        df["Time"] = pd.to_datetime(df["Time"])
        df["id_lot"] = id

        df_full = pd.concat([df_full, df], axis=0)
    
    except:
        print(f'Wrong id: {id}')

43
45
49
50
51
52
53
54
55
56
57
70
72
77
78
79
80
81
82
83
84
85
86
87
104
105
106
107
120
121
122
124
125
126
127
128
129
158
162
163
164
165
166
167
168
169
170
171
172
173
189
190
191
192
194
195
196
198
199
202
207
208
209
210
211
214
215
218
220
237


In [172]:
df_full.head(4)

,Anomaly,Climatology,Crop Cycle,Index,Negative Anomaly,Soil Water Content,Time,id_lot
0,0.050770,0.272558,NaN,0.0,0.0,0.323329,2002-06-16,43
1,0.050232,0.273097,NaN,0.0,0.0,0.323329,2002-06-17,43
2,0.043869,0.272469,NaN,0.0,0.0,0.316338,2002-06-18,43
3,0.031337,0.271660,NaN,0.0,0.0,0.302997,2002-06-19,43


## Obtener polígonos de los lotes en la base de rendimientos

In [228]:
df_aux = df_yields[['longitude', 'latitude']].drop_duplicates()
df_aux['coords'] = list(zip(df_aux['longitude'], df_aux['latitude']))

df_coords = gpd.GeoDataFrame({
    'longitude': list(df_aux['longitude']),
    'latitude': list(df_aux['latitude']),
    'geometry': [Point(p) for p in df_aux['coords']]
}, crs = df_cantones.crs)

df_join = df_coords.sjoin(
    df_cantones,
    how = 'left',
    predicate = 'within'
)

In [229]:
df_coords

,longitude,latitude,geometry
0,-63.48191,-16.36096,POINT (-63.48191 -16.36096)
1,-62.35004,-17.05833,POINT (-62.35004 -17.05833)
2,-62.33751,-17.07043,POINT (-62.33751 -17.07043)
3,-62.40913,-16.92304,POINT (-62.40913 -16.92304)
4,-62.40973,-16.92482,POINT (-62.40973 -16.92482)
...,...,...,...
102,-63.19361,-16.74465,POINT (-63.19361 -16.74465)
103,-63.18130,-16.75335,POINT (-63.1813 -16.75335)
104,-63.16269,-16.77288,POINT (-63.16269 -16.77288)
105,-62.24966,-17.25746,POINT (-62.24966 -17.25746)


In [239]:
df_yields.closest_id_grid

0        0
1        1
2        2
3        3
4        4
      ... 
755    160
756    161
757    161
758    184
759    184
Name: closest_id_grid, Length: 760, dtype: object

In [232]:
df_join.groupby(['departamen','provincia', 'canton']).agg(n = ('ogc_fid', 'count'))

n
departamen provincia           canton                   
Santa Cruz Guarayos            El Puente              29
           Obispo Santiestevan Mineros                13
           Warnes              Los Chacos             10
           Ńuflo de Chavez     San Julian             11
                               San Pedro               9
                               Santa Rosa del Palmar   8
                               Saturnino              27

In [177]:
df_full.merge(
    df_cantones,
    how = 'left',
    left_on = ['id_lot'],
    right_on = ['ogc_fid']
)

,Anomaly,Climatology,Crop Cycle,Index,Negative Anomaly,Soil Water Content,Time,id_lot,ogc_fid,entity,canton,provincia,municipio,departamen,geometry
0,0.050770,0.272558,NaN,0.0,0.0,0.323329,2002-06-16,43,43,Polyline,San Juan del Rosario,Florida,Samaipata,Santa Cruz,"MULTIPOLYGON (((-63.51773 -18.30368, -63.51773..."
1,0.050232,0.273097,NaN,0.0,0.0,0.323329,2002-06-17,43,43,Polyline,San Juan del Rosario,Florida,Samaipata,Santa Cruz,"MULTIPOLYGON (((-63.51773 -18.30368, -63.51773..."
2,0.043869,0.272469,NaN,0.0,0.0,0.316338,2002-06-18,43,43,Polyline,San Juan del Rosario,Florida,Samaipata,Santa Cruz,"MULTIPOLYGON (((-63.51773 -18.30368, -63.51773..."
3,0.031337,0.271660,NaN,0.0,0.0,0.302997,2002-06-19,43,43,Polyline,San Juan del Rosario,Florida,Samaipata,Santa Cruz,"MULTIPOLYGON (((-63.51773 -18.30368, -63.51773..."
4,0.032312,0.270685,NaN,0.0,0.0,0.302997,2002-06-20,43,43,Polyline,San Juan del Rosario,Florida,Samaipata,Santa Cruz,"MULTIPOLYGON (((-63.51773 -18.30368, -63.51773..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572245,0.022881,0.255006,NaN,0.0,0.0,0.277888,2024-10-27,237,237,Polyline,Koque,Loayza,Malla,La Paz,"MULTIPOLYGON (((-67.35134 -17.11373, -67.34945..."
572246,0.025557,0.255502,NaN,0.0,0.0,0.281059,2024-10-28,237,237,Polyline,Koque,Loayza,Malla,La Paz,"MULTIPOLYGON (((-67.35134 -17.11373, -67.34945..."
572247,0.024331,0.254675,NaN,0.0,0.0,0.279006,2024-10-29,237,237,Polyline,Koque,Loayza,Malla,La Paz,"MULTIPOLYGON (((-67.35134 -17.11373, -67.34945..."
572248,0.030046,0.253129,NaN,0.0,0.0,0.283174,2024-10-30,237,237,Polyline,Koque,Loayza,Malla,La Paz,"MULTIPOLYGON (((-67.35134 -17.11373, -67.34945..."


## Example to get data from API

In [88]:
params = {
    "area": 67,  # Climatic area identifier
    "format": "json"  # Output format (optional, default is csv)
}
response = requests.get(API_URL, headers=headers, params=params)
df = pd.json_normalize(response.json())
#df["Time"] = pd.to_datetime(df["Time"])

In [89]:
df.columns

Index(['available_areas', 'error', 'message'], dtype='object')

In [95]:
for el in df.available_areas[0]:
    print(el)

104
105
106
107
120
121
122
124
125
126
127
128
129
158
162
163
164
165
166
167
168
169
170
171
172
173
189
190
191
192
194
195
196
198
199
202
207
208
209
210
211
214
215
218
220
237
43
45
49
50
51
52
53
54
55
56
57
70
72
77
78
79
80
81
82
83
84
85
86
87
